About the Dataset

1) id:unique id for a news article

2) title: the title of a news article

3)author: author of the news article

4) text: the text of the article, colud be incomplete

5) label: a label that marks whether the news article is real or false


1: Fake news

0: real news

Importing the dependencies

In [ ]:
import numpy as np
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [ ]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
# printing the stopword in english
print(stopwords.words('english'))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

Data preprocessing

In [ ]:
# loading the pandas dataset
news_dataset = pd.read_csv('/content/train.csv')

In [ ]:
news_dataset.shape

(20800, 5)

In [ ]:
# print the first five rows of the ataframe
news_dataset.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [ ]:
news_dataset.isnull().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [ ]:
# replacing the null values with null values
news_dataset=news_dataset.fillna('')

In [ ]:
# merging the author name and news title
news_dataset['content']=news_dataset['author']+' '+news_dataset['title']

In [ ]:
print(news_dataset['content'])

0        Darrell Lucus House Dem Aide: We Didn’t Even S...
1        Daniel J. Flynn FLYNN: Hillary Clinton, Big Wo...
2        Consortiumnews.com Why the Truth Might Get You...
3        Jessica Purkiss 15 Civilians Killed In Single ...
4        Howard Portnoy Iranian woman jailed for fictio...
                               ...                        
20795    Jerome Hudson Rapper T.I.: Trump a ’Poster Chi...
20796    Benjamin Hoffman N.F.L. Playoffs: Schedule, Ma...
20797    Michael J. de la Merced and Rachel Abrams Macy...
20798    Alex Ansary NATO, Russia To Hold Parallel Exer...
20799              David Swanson What Keeps the F-35 Alive
Name: content, Length: 20800, dtype: object


In [ ]:
# sepaerating the data and label
x=news_dataset.drop(columns='label',axis=1)
y=news_dataset['label']

In [ ]:
print(x)
print(y)

          id                                              title  \
0          0  House Dem Aide: We Didn’t Even See Comey’s Let...   
1          1  FLYNN: Hillary Clinton, Big Woman on Campus - ...   
2          2                  Why the Truth Might Get You Fired   
3          3  15 Civilians Killed In Single US Airstrike Hav...   
4          4  Iranian woman jailed for fictional unpublished...   
...      ...                                                ...   
20795  20795  Rapper T.I.: Trump a ’Poster Child For White S...   
20796  20796  N.F.L. Playoffs: Schedule, Matchups and Odds -...   
20797  20797  Macy’s Is Said to Receive Takeover Approach by...   
20798  20798  NATO, Russia To Hold Parallel Exercises In Bal...   
20799  20799                          What Keeps the F-35 Alive   

                                          author  \
0                                  Darrell Lucus   
1                                Daniel J. Flynn   
2                             Consortiu

Stemminmg:

stemminmg is the process of reducing the words to its Root word

example:

actor,actress, acting --> act (root word)

In [ ]:
port_stem=PorterStemmer()

In [ ]:
def stemming(content):
  stemmed_content=re.sub('^[a-zA-Z]',' ', content)
  stemmed_content=stemmed_content.lower()
  stemmed_content=stemmed_content.split()
  stemmed_content=[port_stem.stem(word) for word in stemmed_content if not word in stopwords.words('english')]
  stemmed_content=' '.join(stemmed_content)
  return stemmed_content

In [ ]:
news_dataset['content']=news_dataset['content'].apply(stemming)

In [ ]:
print(news_dataset['content'])

0        arrel lucu hous dem aide: didn’t even see come...
1        aniel j. flynn flynn: hillari clinton, big wom...
2                   onsortiumnews.com truth might get fire
3        essica purkiss 15 civilian kill singl us airst...
4        oward portnoy iranian woman jail fiction unpub...
                               ...                        
20795    erom hudson rapper t.i.: trump ’poster child w...
20796    enjamin hoffman n.f.l. playoffs: schedule, mat...
20797    ichael j. de la merc rachel abram macy’ said r...
20798    lex ansari nato, russia hold parallel exercis ...
20799                          avid swanson keep f-35 aliv
Name: content, Length: 20800, dtype: object


In [ ]:
# seperating the data and label
x=news_dataset['content'].values
y=news_dataset['label'].values

In [ ]:
print(x)
print(y)

['rrel lucu hou dem aide: didn’t even see comey’ letter jason chaffetz tweet'
 'niel j. flynn flynn: hillari clinton, big woman campu - breitbart'
 'nsortiumnews.com truth might get fire' ...
 'chael j. de la merc rachel abram macy’ said receiv takeov approach hudson’ bay - new york time'
 'ex ansari nato, russia hold parallel exerci balkan'
 'vid swanson keep f-35 aliv']
[1 0 1 ... 0 1 1]


In [ ]:
y.shape

(20800,)

In [ ]:
# converting textual data to numarical data
vectorizer=TfidfVectorizer()
vectorizer.fit(x)
x=vectorizer.transform(x)

In [ ]:
print(x)

  (0, 19606)	0.26712184655453036
  (0, 17040)	0.23153018548641632
  (0, 11470)	0.32646999159292556
  (0, 11107)	0.2622965941468474
  (0, 10093)	0.3017645735175931
  (0, 9123)	0.20080546193067558
  (0, 6715)	0.2094932520858863
  (0, 5401)	0.2686203362070287
  (0, 5145)	0.2457408363643636
  (0, 4120)	0.22163996553206777
  (0, 3594)	0.3301114684232005
  (0, 1576)	0.32646999159292556
  (0, 843)	0.3565942575980973
  (1, 20976)	0.2989680525520546
  (1, 8888)	0.19783257086654749
  (1, 7388)	0.7101743682057626
  (1, 3949)	0.19027616465467592
  (1, 3303)	0.38636632412207056
  (1, 2924)	0.15813690311212192
  (1, 2484)	0.29227889365065984
  (1, 1228)	0.2668605575019844
  (2, 19525)	0.43255207490270325
  (2, 13619)	0.45601846567592214
  (2, 12246)	0.4870134165836329
  (2, 7919)	0.3430892509912358
  :	:
  (20797, 15674)	0.27017133328848214
  (20797, 15416)	0.261448624169208
  (20797, 13054)	0.07963791048552311
  (20797, 12139)	0.2927512879202997
  (20797, 11578)	0.35834156810969464
  (20797, 10770)

splitting the dataset into traning and test data

In [ ]:
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,stratify=y,random_state=2)

Traning the model: Logistic Regression

In [ ]:
model=LogisticRegression()

In [ ]:
model.fit(x_train,y_train)

LogisticRegression()

Evalution:

acuraccy score

In [ ]:
# accuracy score on the traning data
x_train_prediction=model.predict(x_train)
traning_data_accuracy=accuracy_score(x_train_prediction,y_train)

In [ ]:
print("Accuracy score on the traning data : ",traning_data_accuracy)

Accuracy score on the traning data :  0.9873798076923077


In [ ]:
# accuracy on test data
x_test_prediction=model.predict(x_test)
accuracy_test=accuracy_score(x_test_prediction,y_test)
print("Accuracy score on test data: ",accuracy_test)

Accuracy score on test data:  0.9769230769230769


Making a predictive system

In [ ]:
x_new=x_test[3]
prediction=model.predict(x_new)
print(prediction)
if (prediction[0]==0):
  print("The news is real")
else:
  print("The new is Fake")

[0]
The news is real


In [ ]:
print(y_test[3])

0


In [ ]:
import numpy as np
import pandas as pd
import nltk
import re
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [ ]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
stopwords.words('english')

['i',
 'me',
 'my',
 'myself',
 'we',
 'our',
 'ours',
 'ourselves',
 'you',
 "you're",
 "you've",
 "you'll",
 "you'd",
 'your',
 'yours',
 'yourself',
 'yourselves',
 'he',
 'him',
 'his',
 'himself',
 'she',
 "she's",
 'her',
 'hers',
 'herself',
 'it',
 "it's",
 'its',
 'itself',
 'they',
 'them',
 'their',
 'theirs',
 'themselves',
 'what',
 'which',
 'who',
 'whom',
 'this',
 'that',
 "that'll",
 'these',
 'those',
 'am',
 'is',
 'are',
 'was',
 'were',
 'be',
 'been',
 'being',
 'have',
 'has',
 'had',
 'having',
 'do',
 'does',
 'did',
 'doing',
 'a',
 'an',
 'the',
 'and',
 'but',
 'if',
 'or',
 'because',
 'as',
 'until',
 'while',
 'of',
 'at',
 'by',
 'for',
 'with',
 'about',
 'against',
 'between',
 'into',
 'through',
 'during',
 'before',
 'after',
 'above',
 'below',
 'to',
 'from',
 'up',
 'down',
 'in',
 'out',
 'on',
 'off',
 'over',
 'under',
 'again',
 'further',
 'then',
 'once',
 'here',
 'there',
 'when',
 'where',
 'why',
 'how',
 'all',
 'any',
 'both',
 'each

In [ ]:
data=pd.read_csv('/content/train.csv')

In [ ]:
data.shape

(20800, 5)

In [ ]:
data.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [ ]:
data.isnull().sum()

id        0
title     0
author    0
text      0
label     0
dtype: int64

In [ ]:
data=data.fillna('')

In [ ]:
data['content']=data['author']+' '+data['title']

In [ ]:
data.head()

,id,title,author,text,label,content
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1,Darrell Lucus House Dem Aide: We Didn’t Even S...
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0,"Daniel J. Flynn FLYNN: Hillary Clinton, Big Wo..."
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1,Consortiumnews.com Why the Truth Might Get You...
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1,Jessica Purkiss 15 Civilians Killed In Single ...
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1,Howard Portnoy Iranian woman jailed for fictio...


In [ ]:
x=data['content']
y=data['label']

In [ ]:
print(x)
print(y)

0        Darrell Lucus House Dem Aide: We Didn’t Even S...
1        Daniel J. Flynn FLYNN: Hillary Clinton, Big Wo...
2        Consortiumnews.com Why the Truth Might Get You...
3        Jessica Purkiss 15 Civilians Killed In Single ...
4        Howard Portnoy Iranian woman jailed for fictio...
                               ...                        
20795    Jerome Hudson Rapper T.I.: Trump a ’Poster Chi...
20796    Benjamin Hoffman N.F.L. Playoffs: Schedule, Ma...
20797    Michael J. de la Merced and Rachel Abrams Macy...
20798    Alex Ansary NATO, Russia To Hold Parallel Exer...
20799              David Swanson What Keeps the F-35 Alive
Name: content, Length: 20800, dtype: object
0        1
1        0
2        1
3        1
4        1
        ..
20795    0
20796    0
20797    0
20798    1
20799    1
Name: label, Length: 20800, dtype: int64


In [ ]:
port_stem=PorterStemmer()

In [ ]:

def stemming(content):
  stemmed_content=re.sub('^[a-zA-Z]',' ', content)
  stemmed_content=stemmed_content.lower()
  stemmed_content=stemmed_content.split()
  stemmed_content=[port_stem.stem(word) for word in stemmed_content if not word in stopwords.words('english')]
  stemmed_content=' '.join(stemmed_content)
  return stemmed_content

In [ ]:
data['content']=data['content'].apply(stemming)

In [ ]:
x=data['content'].values
y=data['label'].values

In [ ]:
print(x)
print(y)

['arrel lucu hous dem aide: didn’t even see comey’ letter jason chaffetz tweet'
 'aniel j. flynn flynn: hillari clinton, big woman campu - breitbart'
 'onsortiumnews.com truth might get fire' ...
 'ichael j. de la merc rachel abram macy’ said receiv takeov approach hudson’ bay - new york time'
 'lex ansari nato, russia hold parallel exercis balkan'
 'avid swanson keep f-35 aliv']
[1 0 1 ... 0 1 1]


In [ ]:
vectorizer=TfidfVectorizer()
vectorizer.fit(x)
x=vectorizer.transform(x)

In [ ]:
print(x)

  (0, 19606)	0.26712184655453036
  (0, 17040)	0.23153018548641632
  (0, 11470)	0.32646999159292556
  (0, 11107)	0.2622965941468474
  (0, 10093)	0.3017645735175931
  (0, 9123)	0.20080546193067558
  (0, 6715)	0.2094932520858863
  (0, 5401)	0.2686203362070287
  (0, 5145)	0.2457408363643636
  (0, 4120)	0.22163996553206777
  (0, 3594)	0.3301114684232005
  (0, 1576)	0.32646999159292556
  (0, 843)	0.3565942575980973
  (1, 20976)	0.2989680525520546
  (1, 8888)	0.19783257086654749
  (1, 7388)	0.7101743682057626
  (1, 3949)	0.19027616465467592
  (1, 3303)	0.38636632412207056
  (1, 2924)	0.15813690311212192
  (1, 2484)	0.29227889365065984
  (1, 1228)	0.2668605575019844
  (2, 19525)	0.43255207490270325
  (2, 13619)	0.45601846567592214
  (2, 12246)	0.4870134165836329
  (2, 7919)	0.3430892509912358
  :	:
  (20797, 15674)	0.27017133328848214
  (20797, 15416)	0.261448624169208
  (20797, 13054)	0.07963791048552311
  (20797, 12139)	0.2927512879202997
  (20797, 11578)	0.35834156810969464
  (20797, 10770)

In [ ]:
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,stratify=y,random_state=2)

In [ ]:
model=LogisticRegression()
model.fit(x_train,y_train)

LogisticRegression()

In [ ]:
xp=model.predict(x_train)
xz=accuracy_score(xp,y_train)
print(xz)

0.9873798076923077


In [ ]:
xp=model.predict(x_test)
xz=accuracy_score(xp,y_test)
print(xz)

0.9769230769230769


In [ ]:
input=x_test[2]
x=model.predict(input)
print(x)

[1]
